### This notebook is for adding poster URL link to each movie record.

* Import the latest-updated csv file named imdbProcessed_{date} and run the code starting at the desired index

* When you're done, run through the rest of the code and export the new imdbProcessed dataset with today's date.

In [2]:
import pandas as pd
from datetime import datetime

In [11]:
today = datetime.today()
today = pd.to_datetime(today, format='%Y-%m-%d')
today

Timestamp('2024-10-09 18:58:19.664913')

In [18]:
'''
No need to use this block since we have the initial processing completed.

imdbRaw = pd.read_csv("/Users/ianchang/Library/Mobile Documents/com~apple~CloudDocs/1. Project/Fall 2024/Movie_Creation_Tool-1/ignoredFiles/TMDB_all_movies.csv",usecols=["title", "imdb_id", "release_date", "runtime"])

# Step 1: Convert 'release_date' to datetime and extract the year
imdbRaw['release_date'] = pd.to_datetime(imdbRaw['release_date'], format='%Y-%m-%d', errors='coerce')

# Step 2: Filter rows where both 'imdb_id' and 'release_date' are valid (non-null)
valid_time_rows = imdbRaw[pd.notna(imdbRaw['imdb_id']) & pd.notna(imdbRaw['release_date'])].copy()

# Step 3: Extract the year and assign it to a new column 'release_year'
valid_time_rows['release_year'] = valid_time_rows['release_date'].dt.year

valid_rows = valid_time_rows[valid_time_rows['release_date'] <= datetime.today()]

# Now valid_rows is your "imdbProcessed" dataframe with valid rows and a 'release_year' column
imdbProcessed = valid_rows

imdbProcessed = imdbProcessed.sort_values(by=["release_year"], ascending= False).reset_index(drop=True).drop(columns=["index"])

'''

In [65]:
latest_processed = pd.read_csv("/Users/ianchang/Library/Mobile Documents/com~apple~CloudDocs/1. Project/Fall 2024/Movie_Creation_Tool-1/ians-playground/imdbProcessed_2024-10-10.csv", low_memory= False).drop(columns="Unnamed: 0")

latest_processed

,title,release_date,runtime,imdb_id,release_year,poster_link
0,La Suprema,2024-05-02,83.0,tt20158934,2024,https://m.media-amazon.com/images/M/MV5BYzc0N2...
1,Honeymoon,2024-01-12,90.0,tt19721342,2024,https://m.media-amazon.com/images/M/MV5BN2E1MG...
2,Johnny & Me - A Journey through Time with John...,2024-01-25,104.0,tt15807040,2024,https://m.media-amazon.com/images/M/MV5BNjIwYz...
3,Milk & Serial,2024-08-08,62.0,tt22075376,2024,https://m.media-amazon.com/images/M/MV5BMmVhMD...
4,Burnt Country,2024-02-21,17.0,tt31224230,2024,https://m.media-amazon.com/images/M/MV5BMDdjZW...
...,...,...,...,...,...,...
554394,Sallie Gardner at a Gallop,1878-06-19,1.0,tt2221420,1878,NaN
554395,L'Amazone,1878-05-06,1.0,tt32173050,1878,NaN
554396,"Zim, Boum, Boum",1878-05-07,1.0,tt16763774,1878,NaN
554397,La Glissade,1878-05-06,1.0,tt32172647,1878,NaN


In [21]:
from imdb import Cinemagoer

ia = Cinemagoer()

In [ ]:
import time
lastIndex = 11025

for index, row in latest_processed[lastIndex:].iterrows():
    movieId = row["imdb_id"][2:]
    retries = 3  # Number of retries
    for attempt in range(retries):
        try:
            movie = ia.get_movie(movieId)
            if movie.get("cover url"):
                posterLink = movie.get("cover url")
                latest_processed.loc[index, "poster_link"] = posterLink
            else:
                print(f"Poster not found for movie ID: {movieId}")
            break  # Break the loop if successful

        except Exception as e:
            print(f"Attempt {attempt + 1} failed for movie ID {movieId}: {e}")
            time.sleep(2 ** attempt)  # Exponentially backoff
            if attempt == retries - 1:
                print(f"Max retries reached for movie ID {movieId}. Skipping...")
                continue

In [75]:
latest_processed.loc[11024:,:].head(30)

,title,release_date,runtime,imdb_id,release_year,poster_link
11024,The King of Algiers,2023-05-24,92.0,tt27502506,2023,https://m.media-amazon.com/images/M/MV5BNTA1NG...
11025,The Rapture,2023-10-11,97.0,tt27531109,2023,NaN
11026,This Is It!: The Miracle of BBQ,2023-06-11,100.0,tt29492847,2023,NaN
11027,Our Brothers’ and Sisters’ Keepers,2023-12-09,43.0,tt29964797,2023,NaN
11028,Girl on Girl,2023-09-29,10.0,tt22817866,2023,NaN
11029,The Ethics of Ambiguity,2023-10-01,30.0,tt31022950,2023,NaN
11030,Dublin Crust,2023-09-22,100.0,tt15354254,2023,NaN
11031,We Collide,2023-10-15,2.0,tt29544576,2023,NaN
11032,YDHTLM: You Don't Have to Like Me,2023-10-17,10.0,tt29260667,2023,NaN
11033,HADO,2023-07-10,12.0,tt32241684,2023,NaN


In [76]:
today = datetime.today().date().strftime('%Y-%m-%d')
today

'2024-10-10'

In [77]:
latest_processed.to_csv(f"/Users/ianchang/Library/Mobile Documents/com~apple~CloudDocs/1. Project/Fall 2024/Movie_Creation_Tool-1/ians-playground/imdbProcessed_{today}.csv")